# Example - Estimate Excess Surface Density with stacking strategy

we import some clmm modules

In [1]:
import sys
import os
os.environ['CLMM_MODELING_BACKEND'] = 'nc' # here you may choose ccl or nc (NumCosmo)
sys.path.append('/pbs/throng/lsst/users/cpayerne/CLMM/examples/support')
try: import clmm
except:
    import notebook_install
    notebook_install.install_clmm_pipeline(upgrade=False)
    import clmm
import matplotlib.pyplot as plt
import numpy as np
from numpy import random

clmm.__version__

'0.3.0'

we import `clmm` module's core.

In [2]:
import clmm.polaraveraging as pa
import clmm.galaxycluster as gc
import clmm.modeling as modeling
from clmm import Cosmology 

we import module for mock data

In [3]:
import mock_data as mock

basic modules

In [4]:
import matplotlib.pyplot as plt
import astropy.units as u
import numpy as np
from scipy.optimize import curve_fit

we import `DC2Analysis` modules

In [5]:
import sys
sys.path.append('/pbs/throng/lsst/users/cpayerne/GitForThesis/DC2Analysis')
import stacking as st
import modeling as model
import utils as ut

for reproductibility :

In [6]:
np.random.seed(11)

we define cosmology into `Cosmology` object

In [7]:
mock_cosmo = Cosmology(H0 = 71.0, Omega_dm0 = 0.27 - 0.045, Omega_b0 = 0.045, Omega_k0 = 0.0)

we define the `Modeling` object `moo` to for the galaxy cluster.

In [8]:
moo = clmm.Modeling(massdef = 'mean', delta_mdef = 200, halo_profile_model = 'nfw')
moo.set_cosmo(mock_cosmo)

we define the `Stacking` object `stack` by definning a given range of radial axis, the number of bins for the binned stacked shear profile.

In [9]:
stack = st.Stacking(r_low = 0.5,  r_up = 6, n_bins = 10, cosmo = mock_cosmo)

we now define the type of desired profile : tangential reduced shear of Excess surface density for the stacking procedure by modifying the atribute. We use the method `Type` from the class to select DeltaSigma of reduced tangential shear.

In [10]:
deltasigma = True

In [11]:
stack._select_type(is_deltasigma = deltasigma)

We now define some redshift interval and input mass interval to select GalaxyCluster object in preselected galaxy catalogs from `cosmoDC2`

In [12]:
z_bin = [0.4,0.5]
m_bin = [2*10**14, 2.2*10**14]
m_bin = np.array(m_bin)
n_catalogs =50
ngals = 20000

we define a set of selected cluster with given true masses ans true redshifts

In [13]:
cluster_m = (m_bin[1] - m_bin[0])*np.random.random(n_catalogs) + m_bin[0] #in M_sun
cluster_z = (z_bin[1] - z_bin[0])*np.random.random(n_catalogs) + z_bin[0]

We add each galaxy catalogs that correponds to binning to the `stack` object

In [ ]:
x_list = []

for i in range(n_catalogs):
    
    c = model.Duffy_concentration(cluster_m[i], cluster_z[i], moo)
    
    """Generate mock data"""
    
    data = mock.generate_galaxy_catalog(cluster_m[i], cluster_z[i], c, mock_cosmo,  200,'chang13', zsrc_min = cluster_z[i] + 0.1 ,shapenoise = 0.02,ngals=ngals)

    mock._compute_photoz_pdfs(data, 0.0)
    
    """Create GalaxyCluster object"""
    
    cl = clmm.GalaxyCluster('cluster', 0, 0, cluster_z[i], data)
    
    """Add tangential and cross components"""
    
    # the critical surface density is recquired then set is_deltasigma = True whatever you estimate reduced shear or deltasigma
    
    cl.compute_tangential_and_cross_components(geometry="flat", is_deltasigma = True, cosmo = mock_cosmo)
    
    """Create profile"""
    
    profile = ut.make_gt_profile(cl, stack.r_low, stack.r_up, stack.n_bins, stack.cosmo)
    
    """Estimate individual shear profile"""
    
    stack._estimate_individual_lensing_signal(cl, profile)
    
    x_list.append(profile['radius'])
    
    """Add catalog of background galaxies to stack"""
    
    stack._add_cluster_redshift(cl)
    
    stack._add_background_galaxies(cl, profile)

/pbs/home/c/cpayerne/.conda/envs/mydesc/lib/python3.7/site-packages/numpy/ma/core.py:2832: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  order=order, subok=True, ndmin=ndmin)


we can see how much catalogs have been selected

In [ ]:
print(f'we selected {stack.n_stacked_cluster} clusters')

The attribute `profile` of `shear` object is filled using the method `MakeShearProfile`. We can comput the standard deviation of every lensing signal derived from individial cluster from the stacked lensing signal via `._add_standard_deviation()`

In [ ]:
stack.MakeStackedProfile() 
stack._add_standard_deviation()

The `profile` attribute of `stack`is an astropy table containing the binned profile `gt` along the radial axis `radius` and the error `gt_err`.

In [ ]:
profile = stack.profile

### Visualization of the results

we visualize the reduced tangential stacked shear and we calculate the predicted reduced shear when using the average innput mass of selected clusters. The sum above is aver all lense-source pairs. We also visualize individual shear profiles estimated only by summation over the background galaxies itself.

$$\Delta \Sigma(R) = \left(\frac{1}{\sum\limits_{l,s} w_{l,s}}\right)\sum\limits_{l,s} w_{l,s}\Sigma_c(z_s,z_l)e^t_{l,s}  \ at\ R$$

with : 
    $$
    w_{l,s} = 1/\Sigma_c(z_s,z_l)^2
    $$

we get the estimated shear signal by:

we plot the results for the estimated stacked shear signal (in red) relatively to individual estimated shear signal from all the selected cluster

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16, 6))
plt.style.use('classic')
axes[0].tick_params(axis='both', which='major', labelsize=16)
axes[1].tick_params(axis='both', which='major', labelsize=16)

for i in range(stack.n_stacked_cluster):
    
    axes[0].plot(x_list[i],stack.LS_list[i],c='grey',alpha=0.1)
    
axes[0].plot([],[],c='grey', label='individual profile')    
axes[0].errorbar(profile['radius'], profile['gt'], profile['gt_err'], c='r', fmt = 's', label='stacked profile')
axes[0].set_xlabel('R [Mpc]', fontsize=16)
axes[0].set_ylabel('profile', fontsize=16)
axes[0].legend(fontsize=20, frameon=False)
axes[0].semilogx()
axes[0].semilogy()
#axes[0].set_ylim(10**12,10**14)
axes[0].set_xlim(stack.r_low, stack.r_up)
axes[1].hist(stack.z_galaxy_list, bins = 50,label='histogram', density = True)
axes[1].set_xlabel('redshift', fontsize=16)
axes[1].set_ylabel('galaxy redshift distribution', fontsize=16)
axes[1].legend(fontsize=20, frameon=False)
plt.show()

In [ ]:
def model_data(r, logm):
    
    cluster_z = stack.z_average
    
    z_gal = stack.z_galaxy
    
    c = model.Duffy_concentration(10.**logm, cluster_z, moo)

    if stack.is_deltasigma == False:

        return model.predict_reduced_tangential_shear_z_distrib(r, logm, c, cluster_z , z_gal, moo)

    else:

        return model.predict_excess_surface_density(r, logm, c, stack.z_average, stack.z_galaxy_list,  0,  moo)

## Fitting Halo Mass from Stacked Profile

we can fit the halo mass

In [ ]:
r, gt, gt_err = profile['radius'], profile['gt'], profile['gt_err']

In [ ]:
param_p0 = [14.]
            
param_bounds = ([13. ], [16.])

popt_fit, cov_fit = curve_fit(model_data, r, gt, sigma = gt_err, bounds = param_bounds, p0 = param_p0)

In [ ]:
m = 10**popt_fit[0]
err_m = m*np.log(10)*np.sqrt(cov_fit[0][0])

In [ ]:
print(r'$---> Average input mass M_{200m}^{NFW}$' + f' : {np.mean(cluster_m):2e}' + r'$M_{SUN}$')
print(f'$---> Average fit mass : {m:2e} \pm {err_m:2e}$')

In [ ]:
y_fit = model_data(r, popt_fit[0])
y_model = model_data(r, np.log10(np.mean(cluster_m)))

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16, 6))
plt.style.use('classic')
axes[0].tick_params(axis='both', which='major', labelsize=16)
axes[1].tick_params(axis='both', which='major', labelsize=16)

for i in range(stack.n_stacked_cluster):
    
    axes[0].plot(x_list[i],stack.LS_list[i],c='grey',alpha=0.1)
    
axes[0].plot([],[],c='grey', label='individual profile')    
axes[0].errorbar(profile['radius'], profile['gt'], profile['gt_err'], c = 'r', fmt = 's', ms = 4, label='stacked profile')
axes[0].plot(r, y_fit, label = 'best fit')
axes[0].plot(r, y_model, label = 'model')
axes[0].set_xlabel('R [Mpc]', fontsize=16)
axes[0].set_ylabel('profile', fontsize=16)
axes[0].legend(fontsize=20, frameon=False, loc = 'best', numpoints = 1)
axes[0].semilogx()
axes[0].semilogy()
#axes[0].set_ylim(10**12,10**14)
axes[0].set_xlim(stack.r_low, stack.r_up)
axes[1].hist(stack.z_galaxy_list, bins = 50,label='histogram', density = True)
axes[1].set_xlabel('redshift', fontsize=16)
axes[1].set_ylabel('galaxy redshift distribution', fontsize=16)
axes[1].legend(fontsize=16, frameon=False, loc = 'best')
plt.show()